In [ ]:
from modeling import SimpleCNNRegressor
import torch
from torch import nn, optim

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [ ]:
# モデルの読み込み
def load_model(model_path, device='cpu'):
    from modeling import SimpleCNNRegressor
    import torch
    
    # モデルのインスタンスを作成
    model = SimpleCNNRegressor()
    
    # チェックポイントの読み込み
    checkpoint = torch.load(model_path, map_location='cpu', weights_only=False)
    
    # モデルの重みを読み込み
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # デバイスに移動
    model = model.to(device)
    
    # 評価モードに設定
    model.eval()
    
    # その他の情報も返す
    return model, checkpoint

# 使用例
model, checkpoint = load_model('saved_models/best_model.pth', device=device)
print(f"Loaded model from epoch {checkpoint['epoch']} with val_loss: {checkpoint['best_val_loss']:.6f}")

In [ ]:
import numpy as np
import pandas as pd

def generate_data(df):
    target_indices = []
    timestamps = []
    
    for i in range(checkpoint['window'], len(df)):
        target_indices.append(i)
        timestamps.append(df.iloc[i].name)
        
    return target_indices, timestamps


import os
import joblib
from tqdm import tqdm
from joblib import Parallel, delayed

input_dir = 'history'

def process_file(filename, input_dir):
    """各ファイルを処理する関数"""
    if not filename.endswith('.joblib'):
        return None
    
    file_path = os.path.join(input_dir, filename)
    history_data = joblib.load(file_path)
    
    ticker = history_data['ticker']
    history_df = history_data['history_df']
    
    # データの作成
    target_indices, timestamps = generate_data(history_df)
    
    return {
        'tickers': [ticker] * len(target_indices),
        'target_indices': target_indices,
        'timestamps': timestamps
    }

# 並列処理の実行
files = [f for f in os.listdir(input_dir) if f.endswith('.joblib')]

# n_jobs=-1で全CPUコアを使用
results = Parallel(n_jobs=-1)(
    delayed(process_file)(filename, input_dir) 
    for filename in tqdm(files, desc="Processing files")
)

# 結果の結合
tickers = []
target_indices = []
timestamps = []

for result in results:
    if result is not None:
        tickers.extend(result['tickers'])
        target_indices.extend(result['target_indices'])
        timestamps.extend(result['timestamps'])

In [ ]:
from datetime import datetime, timedelta
is_test = [timestamp >= checkpoint['test_start_timestamp'] for timestamp in timestamps]
test_indices = np.arange(len(tickers))[is_test]

In [ ]:
import numpy as np

def create_image(df):
    size = len(df)
    array = np.zeros((size, size, 3), dtype=np.uint8)

    epsilon = 1e-10
    log_high = np.log(df['High'].values + epsilon)
    log_low = np.log(df['Low'].values + epsilon)
    log_open = np.log(df['Open'].values + epsilon)
    log_close = np.log(df['Close'].values + epsilon)
    
    # NaNを含む時刻のマスクを作成
    nan_mask = (np.isnan(log_high) | np.isnan(log_low) | 
                np.isnan(log_open) | np.isnan(log_close))
    
    global_min = np.nanmin(log_low)
    global_max = np.nanmax(log_high)
    center = (global_min + global_max) / 2

    price_min = center - checkpoint['radius']
    price_max = center + checkpoint['radius']

    # 価格を画像のy座標にマッピング（0が上端で高値、size-1が下端で安値）
    def price_to_y(price):
        # price_maxが上端(0)、price_minが下端(size-1)
        y = (price_max - price) / (price_max - price_min) * (size - 1)
        # NaNの場合は-1を返す（後で無効化するため）
        y = np.where(np.isnan(price), -1, y)
        return np.clip(y, -1, size - 1).astype(int)
    
    # 各時点の価格をy座標に変換
    y_high = price_to_y(log_high)
    y_low = price_to_y(log_low)
    y_open = price_to_y(log_open)
    y_close = price_to_y(log_close)
    
    # x座標（時間軸）のインデックス配列
    x_indices = np.arange(size)
    
    # チャンネル1: HighからLowまでの範囲を255で塗りつぶす
    for i in range(size):
        if not nan_mask[i]:
            # y_high[i]からy_low[i]まで塗りつぶす（y_high <= y_low なので注意）
            array[y_high[i]:y_low[i]+1, i, 0] = 255
    
    # チャンネル2: Open < Closeの場合、OpenからCloseまでを255で塗りつぶす（陽線）
    bullish = log_open < log_close  # 上昇（陽線）
    for i in range(size):
        if bullish[i] and not nan_mask[i]:
            # CloseがOpenより高い（yは小さい）
            y_min = min(y_close[i], y_open[i])  # 上端
            y_max = max(y_close[i], y_open[i])  # 下端
            array[y_min:y_max+1, i, 1] = 255
    
    # チャンネル3: Close < Openの場合、CloseからOpenまでを255で塗りつぶす（陰線）
    bearish = log_close < log_open  # 下降（陰線）
    for i in range(size):
        if bearish[i] and not nan_mask[i]:
            # OpenがCloseより高い（yは小さい）
            y_min = min(y_open[i], y_close[i])  # 上端
            y_max = max(y_open[i], y_close[i])  # 下端
            array[y_min:y_max+1, i, 2] = 255
    
    return array


class Dataset:
    def __init__(self, tickers, target_indices, timestamps, valid_indices, hist_dir='history'):
        self.tickers = tickers
        self.target_indices = target_indices
        self.timestamps = timestamps
        self.hist_dir = hist_dir
        self.valid_indices = valid_indices
        self.window = checkpoint['window']

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        valid_idx = self.valid_indices[idx]
        
        ticker = self.tickers[valid_idx]
        timestamp = self.timestamps[valid_idx].isoformat()
        target_indice = self.target_indices[valid_idx]
        
        history_df = joblib.load(os.path.join(self.hist_dir, f"{ticker}.joblib"))['history_df']
        img = create_image(history_df.iloc[target_indice - self.window: target_indice])
        target_row = history_df.iloc[target_indice]
        target_change = (np.log(target_row['Close'] + 1e-10) - np.log(target_row['Open'] + 1e-10)) * checkpoint['target_change_multiplier']
        # nanの場合は0に置換
        if np.isnan(target_change):
            target_change = 0.0
        return img, target_change, ticker, timestamp

In [ ]:
from torch.utils.data import DataLoader
test_dataset = Dataset(tickers, target_indices, timestamps, test_indices)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8)

In [ ]:
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm

def test_model(model, test_loader, device):
    model.eval()
    
    all_predictions = []
    all_targets = []
    all_tickers = []
    all_timestamps = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            images, targets, tickers_batch, timestamps_batch = batch
            
            # データをデバイスに転送
            images = (images.float().to(device) / 255.0).permute(0, 3, 1, 2)
            targets = targets.float().to(device)
            
            # 予測
            outputs = model(images)
            
            # 結果を保存
            all_predictions.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_tickers.extend(tickers_batch)
            all_timestamps.extend(timestamps_batch)
    
    return np.array(all_predictions), np.array(all_targets), all_tickers, all_timestamps

# テストの実行
predictions, targets, test_tickers, test_timestamps = test_model(model, test_loader, device)

In [ ]:
# In[12]:
# 評価メトリクスの計算
mse = mean_squared_error(targets, predictions)
mae = mean_absolute_error(targets, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(targets, predictions)

print("=" * 50)
print("Test Results:")
print("=" * 50)
print(f"MSE:  {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAE:  {mae:.6f}")
print(f"R2:   {r2:.6f}")

# 相関係数
correlation = np.corrcoef(targets, predictions[:, 0])[0, 1]
print(f"Correlation: {correlation:.6f}")

In [ ]:
# In[13]:
# 結果の可視化
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. 予測値 vs 実際値の散布図
ax = axes[0, 0]
ax.scatter(targets, predictions, alpha=0.3, s=1)
ax.plot([targets.min(), targets.max()], [targets.min(), targets.max()], 'r--', lw=2)
ax.set_xlabel('Actual Change')
ax.set_ylabel('Predicted Change')
ax.set_title(f'Predictions vs Actual (R2={r2:.4f})')
ax.grid(True, alpha=0.3)

# 2. 残差プロット
residuals = predictions - targets
ax = axes[0, 1]
ax.scatter(predictions, residuals, alpha=0.3, s=1)
ax.axhline(y=0, color='r', linestyle='--')
ax.set_xlabel('Predicted Change')
ax.set_ylabel('Residuals')
ax.set_title('Residual Plot')
ax.grid(True, alpha=0.3)

# 3. 残差のヒストグラム
ax = axes[1, 0]
ax.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('Residuals')
ax.set_ylabel('Frequency')
ax.set_title(f'Distribution of Residuals (std={np.std(residuals):.6f})')
ax.grid(True, alpha=0.3)

# 4. 予測値と実際値のヒストグラム
ax = axes[1, 1]
ax.hist(targets, bins=50, alpha=0.5, label='Actual', color='blue', edgecolor='black')
ax.hist(predictions, bins=50, alpha=0.5, label='Predicted', color='red', edgecolor='black')
ax.set_xlabel('Change')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Actual vs Predicted')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# In[14]:
# 銘柄別のパフォーマンス分析
results_df = pd.DataFrame({
    'ticker': test_tickers,
    'timestamp': test_timestamps,
    'actual': targets,
    'predicted': predictions,
    'residual': residuals,
    'abs_error': np.abs(residuals)
})

# 銘柄ごとの統計
ticker_stats = results_df.groupby('ticker').agg({
    'abs_error': ['mean', 'std', 'count'],
    'residual': ['mean', 'std'],
    'actual': 'std',
    'predicted': 'std'
}).round(6)

ticker_stats.columns = ['_'.join(col).strip() for col in ticker_stats.columns]
ticker_stats = ticker_stats.sort_values('abs_error_mean')

print("\n" + "=" * 50)
print("Top 10 Best Performing Tickers:")
print("=" * 50)
print(ticker_stats.head(10))

print("\n" + "=" * 50)
print("Top 10 Worst Performing Tickers:")
print("=" * 50)
print(ticker_stats.tail(10))

In [ ]:
# In[15]:
# 時系列でのパフォーマンス分析
results_df['timestamp'] = pd.to_datetime(results_df['timestamp'])
results_df = results_df.sort_values('timestamp')

# 日次のパフォーマンス
daily_performance = results_df.groupby(results_df['timestamp'].dt.date).agg({
    'abs_error': 'mean',
    'residual': ['mean', 'std'],
    'actual': 'count'
}).round(6)

daily_performance.columns = ['_'.join(col).strip() if col[1] else col[0] for col in daily_performance.columns]

# 時系列プロット
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

ax = axes[0]
ax.plot(daily_performance.index, daily_performance['abs_error'], label='MAE', color='blue')
ax.set_xlabel('Date')
ax.set_ylabel('Mean Absolute Error')
ax.set_title('Daily Performance')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.bar(daily_performance.index, daily_performance['actual'], label='Sample Count', color='green', alpha=0.5)
ax.set_xlabel('Date')
ax.set_ylabel('Number of Predictions')
ax.set_title('Daily Sample Count')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()